# Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

RANDOM_STATE = 12345

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Пропусков нет, все признаки количественные. Пропущенных значений нет. Для удобства переименуем название столбцов и сменим тип колонки возраст на int.

In [3]:
df.columns = ['sex', 'age', 'salary', 'fam_member', 'ins_payments']
df['age'] = df['age'].astype('int')

In [4]:
df.duplicated().sum()

153

Дубликаты есть, но возможно это просто совпадение. Было бы неплохо выяснить это у заказчика или посмотреть в источнике данных. Удалять их не будем. Оставим как есть.

Проверим изменения.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
sex             5000 non-null int64
age             5000 non-null int64
salary          5000 non-null float64
fam_member      5000 non-null int64
ins_payments    5000 non-null int64
dtypes: float64(1), int64(4)
memory usage: 195.4 KB


## Умножение матриц

Ответим на вопрос: изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:** Запишем формулу предсказания, умножив признаки на обратимую матрицу P.

$$
a = XP ((XP)^T XP)^{-1} (XP)^T y = XP (P^T X^T XP)^{-1} P^T X^T y = XP (X^T XP)^{-1} E X^T y = XP P^{-1} (X^T X)^{-1} E X^T y = X E (X^T X)^{-1} E X^T y = X (X^T X)^{-1} X^T y = Xw
$$

Итак, произведя вычисления мы вернулись к изначальной формуле: $a = Xw$. Значит, если признаки умножить на обратимую матрицу, то качество линейной регрессии не уменьшится.

## Алгоритм преобразования

**Алгоритм**

1. Создадим случайную матрицу Р.
2. Найдем обратную матрицу. Если обратной матрицы нет, то создадим матрицу Р еще раз.
3. Умножим матрицу признаков Х на новую обратимую матрицу Р.
4. Обучим модель линейной регрессии. Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применим метрику R2.

**Обоснование**

Мы уже провели исследование о том, что качество линейной регрессии не поменяется при умножении признаков на обратимую матрицу.

## Проверка алгоритма

Разделим данные на признаки и целевой признак. Затем преобразуем в матрицы.

In [6]:
features = df.drop('ins_payments', axis=1)
target = df['ins_payments']
print('features', features.shape)
print('target', target.shape)

features (5000, 4)
target (5000,)


In [7]:
features_matrix = features.values
features_matrix

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

Создадим случайную матрицу и найдем обратную матрицу.

In [8]:
random_matrix = np.random.normal(size=(features.shape[1], features.shape[1]))
random_matrix

array([[-1.14958471, -1.7256397 , -0.22665026,  0.11520417],
       [ 0.29280739,  0.79937121, -0.33946302,  1.22553979],
       [ 1.64177525, -0.52125917,  2.25074018,  0.30763347],
       [-1.64587438,  0.37828483, -1.94171715,  0.22848491]])

Найдем обратную матрицу для случайной.

In [9]:
np.linalg.inv(random_matrix)

array([[ 0.78047806,  1.51582604, -3.25993705, -4.13486268],
       [-0.97619567, -0.79054767,  1.78817182,  2.32491477],
       [-0.82567454, -1.3694462 ,  3.16932425,  3.49450544],
       [ 0.22155706,  0.59012334,  0.49038715,  0.43940131]])

Случайная матрица обратима. Переходим к следующему шагу: умножим матрицу признаков на случайную матрицу.

In [10]:
features_matrix_new = features_matrix @ random_matrix
features_matrix_new

array([[ 81441.26211576, -25823.02795342, 111620.6267453 ,
         15309.21072588],
       [ 62399.2828206 , -19770.69908896,  85510.56995304,
         11746.67501705],
       [ 34485.77169472, -10923.26079909,  47255.69942377,
          6495.8434365 ],
       ...,
       [ 55658.74542316, -17653.94185964,  76289.41952249,
         10453.74225738],
       [ 53686.4052771 , -17028.17946844,  73585.68400896,
         10087.37686706],
       [ 66661.47835666, -21142.08725164,  91368.37811398,
         12524.57751607]])

Теперь обучим модель на не преобразованных данных. Но для начала разделим данные на две выборки: обучающую и валидационную.

In [11]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              train_size=0.75, 
                                                                              test_size=0.25, 
                                                                              random_state=RANDOM_STATE)

Проверим, правильно ли мы поделили данные.

In [12]:
print('Размер выборки features_train:', features_train.shape)
print('Размер выборки features_valid:', features_valid.shape)
print('Размер выборки target_train:', target_train.shape)
print('Размер выборки target_valid:', target_valid.shape)

Размер выборки features_train: (3750, 4)
Размер выборки features_valid: (1250, 4)
Размер выборки target_train: (3750,)
Размер выборки target_valid: (1250,)


Обучим модель на не преобразованных данных.

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
r2_first = r2_score(target_valid, predictions)
print('Метрика R2 до преобразования данных:', r2_first)

Метрика R2 до преобразования данных: 0.435227571270266


Метрика не слишком высокая. Теперь сделаем то же самое, но с преобразованными данными.

In [14]:
features_train_transform, features_valid_transform, target_train_transform, target_valid_transform = train_test_split(
    features_matrix_new, target, train_size=0.75, test_size=0.25, random_state=RANDOM_STATE)

Проверим, правильно ли мы поделили данные.

In [15]:
print('Размер выборки features_train_transform:', features_train_transform.shape)
print('Размер выборки features_valid_transform:', features_valid_transform.shape)
print('Размер выборки target_train_transform:', target_train_transform.shape)
print('Размер выборки target_valid_transform:', target_valid_transform.shape)

Размер выборки features_train_transform: (3750, 4)
Размер выборки features_valid_transform: (1250, 4)
Размер выборки target_train_transform: (3750,)
Размер выборки target_valid_transform: (1250,)


Обучим модель на преобразованных данных.

In [16]:
model = LinearRegression()
model.fit(features_train_transform, target_train_transform)
predictions = model.predict(features_valid_transform)
r2_transform = r2_score(target_valid_transform, predictions)
print('Метрика R2 до преобразования данных:', r2_transform)

Метрика R2 до преобразования данных: 0.435227571271704


Результат метрики схож с предыдущим.

In [17]:
print('Метрика R2 до преобразования данных:', r2_first)
print('Метрика R2 после преобразования данных:', r2_transform)

Метрика R2 до преобразования данных: 0.435227571270266
Метрика R2 после преобразования данных: 0.435227571271704


Метрики R2 почти равны. Значит, что качество линейной регрессии из sklearn не отличается до и после преобразования. 